In [1]:
#Import dependencies
import pandas as pd
import gmaps
import requests
from config import g_key

# Configure gmaps to use your Google API key.

gmaps.configure(api_key=g_key)

In [2]:
city_data_df=pd.read_csv("Challenge/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches,Snow inches
0,0,Powell River,CA,2020-02-28 05:25:03,49.85,-124.54,46.40,87,90,9.17,Clouds,0.00,0.0
1,1,Basco,PH,2020-02-28 05:28:03,20.45,121.97,75.25,82,40,18.88,Rain,1.13,0.0
2,2,Nara,JP,2020-02-28 05:28:03,34.69,135.80,51.01,39,75,2.24,Clouds,0.00,0.0
3,3,Narsaq,GL,2020-02-28 05:28:04,60.92,-46.05,6.80,42,0,3.36,Clear,0.00,0.0
4,4,Kaitangata,NZ,2020-02-28 05:28:04,-46.28,169.85,75.00,69,79,4.00,Clouds,0.00,0.0


In [3]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

# Ask the customer the following:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_true = input("Do you want it to be raining? (yes/no) ")
snow_true = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 50
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
if rain_true == "yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Current Description"] == "Rain")]
elif snow_true == "yes": 
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Current Description"] == "Snow")]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches,Snow inches
0,0,Powell River,CA,2020-02-28 05:25:03,49.85,-124.54,46.40,87,90,9.17,Clouds,0.0,0.0
7,7,Smithers,CA,2020-02-28 05:27:31,54.78,-127.17,35.60,80,90,12.75,Clouds,0.0,0.0
12,12,Kodiak,US,2020-02-28 05:28:07,57.79,-152.41,30.20,74,1,9.17,Clear,0.0,0.0
14,14,Ushuaia,AR,2020-02-28 05:27:36,-54.80,-68.30,46.40,75,40,38.03,Clouds,0.0,0.0
24,24,Nikolskoye,RU,2020-02-28 05:28:12,59.70,30.79,28.40,79,90,6.71,Clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,507,Lyngseidet,NO,2020-02-28 05:31:27,69.58,20.22,26.60,48,92,21.92,Clouds,0.0,0.0
508,508,Napoleon,US,2020-02-28 05:31:27,41.39,-84.13,21.00,85,1,5.82,Clear,0.0,0.0
512,512,Tallahassee,US,2020-02-28 05:26:55,30.44,-84.28,41.00,64,75,4.70,Clouds,0.0,0.0
513,513,Murfreesboro,US,2020-02-28 05:31:29,35.85,-86.39,41.00,51,90,3.36,Clouds,0.0,0.0


In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates and add Hotel Name
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#Searching for a hotel
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][1]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No Hotel around found"
        pass
    
output_data_file = "Challenge/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label="City_ID")

hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Powell River,CA,46.40,Clouds,49.85,-124.54,Marine Inn
7,Smithers,CA,35.60,Clouds,54.78,-127.17,Stork Nest Inn
12,Kodiak,US,30.20,Clear,57.79,-152.41,Best Western Kodiak Inn and Convention Center
14,Ushuaia,AR,46.40,Clouds,-54.80,-68.30,Las Lengas Hotel
24,Nikolskoye,RU,28.40,Clouds,59.70,30.79,Gostinitsa Dlya Krupnogabaritnykh Gruzov
...,...,...,...,...,...,...,...
507,Lyngseidet,NO,26.60,Clouds,69.58,20.22,Kjelkebakken Gard
508,Napoleon,US,21.00,Clear,41.39,-84.13,Knights Inn Napoleon
512,Tallahassee,US,41.00,Clouds,30.44,-84.28,DoubleTree by Hilton Hotel Tallahassee
513,Murfreesboro,US,41.00,Clouds,35.85,-86.39,DoubleTree by Hilton Hotel Murfreesboro


In [6]:
#Template for the pop-up marker
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""


hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Calling the figure
fig

Figure(layout=FigureLayout(height='420px'))